In [ ]:
import re
import pickle
import nltk
from nltk.tokenize import  TweetTokenizer
from nltk.corpus import stopwords

In [ ]:
from nltk.stem.snowball import SnowballStemmer

tokenizer =  TweetTokenizer()
stop = set(stopwords.words('english')+['<user>','<url>',',','-','.','[',']',':','&'])
stemmer = SnowballStemmer("english");
mojitable = pickle.load(open('emojtable.😃', 'rb'))
mojitable = {k: v for k, v in mojitable.items() if v != ""}
keys = [k for k, v in mojitable.items()]
prog = re.compile("(?:\s|^)((?:[^\s][^a-zA-Z1-9\n]+)(?:[^\s](?:[^a-zA-Z1-9\n]+|$))+)")
lineHead = re.compile("^(\d+,)");

def replaceMoji(s):
    def checkSubstring(emo):
        for k in keys:
            if k in emo :
                return mojitable[k]
        return emo;
    splitted = prog.split(s)
    moj = ""
    nomoj = ""
    for i in range(0,int(len(splitted)/2)):
        nomoj += splitted[2*i]+" "
        emo = splitted[2*i+1].strip()
        parsedEmo = mojitable.get(emo,checkSubstring(emo))
        if parsedEmo == "":
            parsedEmo = emo
        moj += " "+parsedEmo+" "    
    return (moj,nomoj+splitted[-1]);
def prepareOneLine(line):
    try :
        tweet = line.decode('utf-8')
    except UnicodeDecodeError :
        tweet = str(line)[2:-5]
        print("cant parse "+tweet)
    head = lineHead.search(tweet)
    header = ""
    if head != None :
        header = head.group(1)
        tweet = tweet[len(header):]
    return header+preparOneTweets(tweet)
            
def preparOneTweets(tweet):
    moj,nomoj = replaceMoji(tweet)
    return  " ".join([stemmer.stem(i) for i in tokenizer.tokenize(nomoj) if i not in stop])+" "+moj
def preprocessFile(file):
    lines = []
    with open(file,'rb') as f :
        lines = map(prepareOneLine,f.readlines())
        f.close()
    with open("cleaned"+file,"w") as f:
        f.write("\n".join(list(lines)))
        f.close()
        print("done "+file)

preparOneTweets("took pictur donkey rectum ( ( ( / issuestemporaryrestrainingorderaswel /")

In [ ]:
files = ["train_neg.txt","train_neg_full.txt","train_pos.txt","train_pos_full.txt","test_data.txt"]
for f in files:
    preprocessFile(f)